In [ ]:
import pandas as pd
import os
import bs4
import json
import requests
from datetime import timedelta, date
from bs4 import BeautifulSoup

In [ ]:
base_url = 'https://codigospostalessitecorecms.azurewebsites.net/Codigo-Postal/'
getMunicipality = 'wApi/getMunicipality'
getSettlement='wApi/getSettlement'

In [ ]:
f = open('../output/Chiapas_houses.json',)
data = json.load(f)

# f = open('../../../EDOMEX_apartments.json')
# data = json.load(f)

# df = pd.read_json('../../../EDOMEX_apartments.json')
# df.head()


In [ ]:
type(data[0])

In [ ]:
try:
    for element in df:
        estado = element['Estado']
        ciudad = element['Ciudad'].upper()
        colonia= element['Colonia'].upper()
        
        first_page = requests.get(base_url)
        html = bs4.BeautifulSoup(first_page.text, "html5lib")
        select = html.find('select', attrs={'id':'cve_state'})
        options = select.findAll('option')
        
        for option in options:
            if option.text == estado:
                cve_estado = option['value']
                print(f'Estado {cve_estado}')
                
                payload = {"cve_state":cve_estado,\
                           "full":"true"}
                second_page = requests.post(base_url+getMunicipality, data=payload)
                html = bs4.BeautifulSoup(second_page.text, "html5lib")
                select = html.find('select', attrs={'id':'cve_municipality'})
                options = select.findAll('option')
                
                for option in options:
                    #print(f'{option.text}-{ciudad}')
                    if option.text.upper() == ciudad:
                        cve_municipality = option['value']
                        print(f'Ciudad {cve_municipality}')
                        
                        payload = {"cve_state":cve_estado,\
                                   "cve_municipality":cve_municipality}
                        third_page = requests.post(base_url+getMunicipality, data=payload)
                        html = bs4.BeautifulSoup(third_page.text, "html5lib")
                        select = html.find('select', attrs={'id':'cve_settlement'})
                        options = select.findAll('option')

                        for option in options:
                            #print(f'{option.text}-{colonia}')
                            if option.text.upper() == colonia:
                                cve_settlement = option['value']
                                print(f'Colonia {cve_settlement}')

                                payload = {"cve_state":cve_estado,\
                                           "cve_municipality":cve_municipality,\
                                           "cve_settlement":cve_settlement}
                                fourth_page = requests.post(base_url+getMunicipality, data=payload)
                                html = bs4.BeautifulSoup(fourth_page.text, "html5lib")
                                tr = html.find('tr', class_='pagedemo _current')
                                codigo_postal = tr.find('td').text
                                print(f'CP {codigo_postal}')
                                print('-------------')
                                element['cp'] = codigo_postal
                
except:
        print(element)

In [ ]:
data

In [ ]:
element